In [1]:
# Test the Pyppeteer Puppeteer wrapper


# References
# - https://scrapeops.io/python-web-scraping-playbook/python-pyppeteer/
# - Note Chromium installed at /Users/stephengodfrey/Library/Application Support/pyppeteer/local-chromium/1181205


In [4]:
import asyncio
from pyppeteer import launch
from bs4 import BeautifulSoup


In [5]:

# Get a screenshot of page
# async def main():
#     browser = await launch()
#     page = await browser.newPage()
#     await page.goto('https://quotes.toscrape.com/')
#     await page.screenshot({'path': 'screenshot.png'})
#     await browser.close()

# Get the html content and view with beautiful soup
async def main():
    browser = await launch()
    page = await browser.newPage()
    await page.goto('https://www.cccco.edu/')

    ## Get HTML
    html = await page.content()
    await browser.close()
    return html

html_response = await main()

## Load HTML Response Into BeautifulSoup
soup = BeautifulSoup(html_response, "html.parser")
# title = quote_block.find('h1').text
# print('title', title)






In [6]:
soup
# Return all links
page_urls = []
for a in soup.find_all('a', href=True):
    # print("Found the URL:", a['href'])
    # page_urls.append((a.contents, a['href']))
    page_urls.append((a['href']))


In [7]:
page_urls

['#page-banner',
 '/Students/Find-a-College',
 'https://home.cccapply.org/en/',
 '/College-Professionals/Data',
 '/About-Us/News-and-Media',
 '/About-Us/job-opportunities',
 '/About-Us/Contact',
 '/',
 '/',
 '/Students',
 '/Students/Find-a-College',
 '/Students/Apply-for-College',
 '/Students/Pay-for-College',
 '/Students/Transfer',
 '/Students/Career-Education',
 '/Students/Support-Services',
 '/About-Us/Notable-Alumni',
 '/College-Professionals',
 '/College-Professionals/Data',
 '/College-Professionals/Guided-Pathways',
 '/About-Us/Chancellors-Office/Divisions',
 '/Search-Memos-and-Official-Documents',
 'https://cccpln.csod.com/LMS/catalog/Welcome.aspx',
 '/About-Us',
 '/About-Us/About-California-Community-Colleges',
 '/About-Us/Vision-2030',
 '/About-Us/State-of-the-System',
 '/About-Us/Key-Facts',
 '/About-Us/Chancellors-Office',
 '/About-Us/Board-of-Governors',
 '/About-Us/Consultation-Council',
 '/About-Us/News-and-Media',
 '/About-Us/Reports',
 '/About-Us/diversity-equity-inclus

In [20]:
dir(a)

['DEFAULT_INTERESTING_STRING_TYPES',
 'EMPTY_ELEMENT_EVENT',
 'END_ELEMENT_EVENT',
 'START_ELEMENT_EVENT',
 'STRING_ELEMENT_EVENT',
 '__bool__',
 '__call__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '__weakref__',
 '_all_strings',
 '_clone',
 '_event_stream',
 '_find_all',
 '_find_one',
 '_format_tag',
 '_indent_string',
 '_is_xml',
 '_lastRecursiveChild',
 '_last_descendant',
 '_namespaces',
 '_should_pretty_print',
 'append',
 'attrs',
 'can_be_empty_element',
 'cdata_list_attributes',
 'childGenerator',
 'chi

In [22]:
a.get_text()

'Complaints'

In [25]:
type(a)
dir(a)
a.contents


['Complaints']